<a href="https://colab.research.google.com/github/hsong-77/transformer-practice/blob/main/few-to-no-labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install nlpaug==1.1.7
!pip install scikit-multilearn==0.2.0
!pip install faiss-cpu

In [ ]:
import pandas as pd

dataset_url = "https://git.io/nlp-with-transformers"
df_issues = pd.read_json(dataset_url, lines=True)
df_issues.shape

In [ ]:
cols = ["url", "id", "title", "user", "labels", "state", "created_at", "body"]
df_issues.loc[2, cols].to_frame()

In [ ]:
df_issues["labels"] = df_issues["labels"].apply(lambda x: [meta["name"] for meta in x])
df_issues["labels"].head()

In [ ]:
df_issues["labels"].apply(lambda x: len(x)).value_counts().to_frame().T

In [ ]:
df_counts = df_issues["labels"].explode().value_counts()
print(f"Number of labels: {len(df_counts)}")
df_counts.to_frame().head(8).T

In [ ]:
label_map = {"Core: Tokenization": "tokenization",
             "New model": "new model",
             "Core: Modeling": "model training",
             "Usage": "usage",
             "Core: Pipeline": "pipeline",
             "TensorFlow": "tensorflow or tf",
             "PyTorch": "pytorch",
             "Examples": "examples",
             "Documentation": "documentation"}

def filter_labels(x):
  return [label_map[label] for label in x if label in label_map]

df_issues["labels"] = df_issues["labels"].apply(filter_labels)
all_labels = list(label_map.values())

In [ ]:
df_counts = df_issues["labels"].explode().value_counts()
print(f"Number of labels: {len(df_counts)}")
df_counts.to_frame().T

In [ ]:
df_issues["split"] = "unlabeled"
mask = df_issues["labels"].apply(lambda x: len(x)) > 0
df_issues.loc[mask, "split"] = "labeled"
df_issues["split"].value_counts().to_frame()

In [ ]:
for column in ["title", "body", "labels"]:
  print(f"{column}: {df_issues[column].iloc[26][:500]}\n")

In [ ]:
df_issues["text"] = df_issues.apply(lambda x: x["title"] + "\n\n" + x["body"], axis=1)

len_before = len(df_issues)
df_issues = df_issues.drop_duplicates(subset="text")
print(f"Removed {(len_before-len(df_issues))/len_before:.2%} duplicates.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df_issues["text"].str.split().apply(len).hist(bins=np.linspace(0, 500, 50), grid=False, edgecolor="C0")
plt.title("Words per issue")
plt.xlabel("Number of words")
plt.ylabel("Number of issues")
plt.show()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()
mlb.fit([all_labels])

def balanced_split(df, test_size=0.5):
  labels = mlb.transform(df["labels"])
  ind = np.expand_dims(np.arange(len(df)), axis=1)
  ind_train, _, ind_test, _ = iterative_train_test_split(ind, labels, test_size)

  return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

df_clean = df_issues[["text", "labels", "split"]].reset_index(drop=True).copy()
df_unsup = df_clean.loc[df_clean["split"]=="unlabeled", ["text", "labels"]]
df_sup = df_clean.loc[df_clean["split"]=="labeled", ["text", "labels"]]

np.random.seed(0)
df_train, df_tmp = balanced_split(df_sup, test_size=0.5)
df_valid, df_test = balanced_split(df_tmp, test_size=0.5)

ds = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "valid": Dataset.from_pandas(df_valid.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
    "unsup": Dataset.from_pandas(df_unsup.reset_index(drop=True))})
ds

In [ ]:
np.random.seed(0)
all_indices = np.expand_dims(list(range(len(ds["train"]))), axis=1)
indices_pool = all_indices
labels = mlb.transform(ds["train"]["labels"])
train_samples = [8, 16, 32, 64, 128]
train_slices, last_k = [], 0

for i, k in enumerate(train_samples):
  indices_pool, labels, new_slice, _ = iterative_train_test_split(indices_pool, labels, (k-last_k)/len(labels))
  last_k = k
  if i == 0: train_slices.append(new_slice)
  else: train_slices.append(np.concatenate((train_slices[-1], new_slice)))

train_slices.append(all_indices)
train_samples.append(len(ds["train"]))
train_slices = [np.squeeze(train_slice) for train_slice in train_slices]

In [ ]:
print("Target split sizes:")
print(train_samples)
print("Actual split sizes:")
print([len(x) for x in train_slices])

In [ ]:
# naive bayes
def prepare_labels(batch):
  batch["label_ids"] = mlb.transform(batch["labels"])
  return batch

ds = ds.map(prepare_labels, batched=True)

In [ ]:
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer

macro_scores, micro_scores = defaultdict(list), defaultdict(list)

for train_slice in train_slices:
  # get data
  ds_train_sample = ds["train"].select(train_slice)
  y_train = np.array(ds_train_sample["label_ids"])
  y_test = np.array(ds["test"]["label_ids"])
  # encode
  count_vect = CountVectorizer()
  X_train_counts = count_vect.fit_transform(ds_train_sample["text"])
  X_test_counts = count_vect.transform(ds["test"]["text"])
  # train
  classifier = BinaryRelevance(classifier=MultinomialNB())
  classifier.fit(X_train_counts, y_train)
  # predict and evaluate
  y_pred_test = classifier.predict(X_test_counts)
  clf_report = classification_report(y_test, y_pred_test, target_names=mlb.classes_, zero_division=0, output_dict=True)
  # metrics
  macro_scores["Naive Bayes"].append(clf_report["macro avg"]["f1-score"])
  micro_scores["Naive Bayes"].append(clf_report["micro avg"]["f1-score"])

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(micro_scores, macro_scores, sample_sizes, current_model):
  fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

  for run in micro_scores.keys():
    if run == current_model:
      ax0.plot(sample_sizes, micro_scores[run], label=run, linewidth=2)
      ax1.plot(sample_sizes, macro_scores[run], label=run, linewidth=2)
    else:
      ax0.plot(sample_sizes, micro_scores[run], label=run, linestyle="dashed")
      ax1.plot(sample_sizes, macro_scores[run], label=run, linestyle="dashed")

  ax0.set_title("Micro F1 scores")
  ax1.set_title("Macro F1 scores")
  ax0.set_ylabel("Test set F1 score")
  ax0.legend(loc="lower right")
  for ax in [ax0, ax1]:
    ax.set_xlabel("Number of training samples")
    ax.set_xscale("log")
    ax.set_xticks(sample_sizes)
    ax.set_xticklabels(sample_sizes)
    ax.minorticks_off()
  
  plt.tight_layout()
  plt.show()

In [ ]:
plot_metrics(micro_scores, macro_scores, train_samples, "Naive Bayes")

In [ ]:
# zero-shot
from transformers import pipeline

# prompt
pipe = pipeline("fill-mask", model="bert-base-uncased")

In [ ]:
movie_desc = "The main characters of the movie madacascar are a lion, a zebra, a giraffe, and a hippo. "
prompt = "The movie is about [MASK]."

output = pipe(movie_desc + prompt, targets=["animals", "cars"])
for element in output:
  print(f"Token {element['token_str']}:\t{element['score']:.3f}%")

In [ ]:
# test entailment
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# pipe = pipeline("zero-shot-classification", model="facebook/bart-base")

In [ ]:
def zero_shot_pipeline(example):
  output = pipe(example["text"], all_labels, multi_label=True)
  example["predicted_labels"] = output["labels"]
  example["scores"] = output["scores"]
  return example

ds_zero_shot = ds["valid"].map(zero_shot_pipeline)

In [ ]:
def get_preds(example, threshold=None, topk=None):
  preds = []
  if threshold:
    for label, score in zip(example["predicted_labels"], example["scores"]):
      if score >= threshold:
        preds.append(label)
  elif topk:
    for i in range(topk):
      preds.append(example["predicted_labels"][i])
  else:
    raise ValueError("Set either `threshold` or `topk`.")

  return {"pred_label_ids": list(np.squeeze(mlb.transform([preds])))}

In [ ]:
def get_clf_report(ds):
  y_true = np.array(ds["label_ids"])
  y_pred = np.array(ds["pred_label_ids"])
  return classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

In [ ]:
macros, micros = [], []
topks = [1, 2, 3, 4]

for topk in topks:
  ds_zero_shot = ds_zero_shot.map(get_preds, batched=False, fn_kwargs={"topk": topk})
  clf_report = get_clf_report(ds_zero_shot)
  micros.append(clf_report['micro avg']['f1-score'])
  macros.append(clf_report['macro avg']['f1-score'])

In [ ]:
plt.plot(topks, micros, label='Micro F1')
plt.plot(topks, macros, label='Macro F1')
plt.xlabel("Top-k")
plt.ylabel("F1-score")
plt.legend(loc='best')
plt.show()

In [ ]:
macros, micros = [], []
thresholds = np.linspace(0.01, 1, 100)

for threshold in thresholds:
  ds_zero_shot = ds_zero_shot.map(get_preds, fn_kwargs={"threshold": threshold})
  clf_report = get_clf_report(ds_zero_shot)
  micros.append(clf_report["micro avg"]["f1-score"])
  macros.append(clf_report["macro avg"]["f1-score"])

In [ ]:
plt.plot(thresholds, micros, label="Micro F1")
plt.plot(thresholds, macros, label="Macro F1")
plt.xlabel("Threshold")
plt.ylabel("F1-score")
plt.legend(loc="best")
plt.show()

In [ ]:
ds_zero_shot = ds["test"].map(zero_shot_pipeline)
ds_zero_shot = ds_zero_shot.map(get_preds, fn_kwargs={"topk": 1})
clf_report = get_clf_report(ds_zero_shot)

for train_slice in train_slices:
  macro_scores['Zero Shot'].append(clf_report['macro avg']['f1-score'])
  micro_scores['Zero Shot'].append(clf_report['micro avg']['f1-score'])

plot_metrics(micro_scores, macro_scores, train_samples, "Zero Shot")

In [ ]:
# few-shot
# data augmentation
from transformers import set_seed
import nlpaug.augmenter.word as naw

set_seed(3)
aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-uncased", device="cpu", action="substitute")

In [ ]:
text = "Transformers are the most popular toys"
print(f"Original text: {text}")
print(f"Augmented text: {aug.augment(text)}")

In [ ]:
def augment_text(batch, transformations_per_example=1):
  text_aug, label_ids = [], []
  for text, labels in zip(batch["text"], batch["label_ids"]):
    text_aug += [text]
    label_ids += [labels]
    for _ in range(transformations_per_example):
      text_aug += [aug.augment(text)]
      label_ids += [labels]

  return {"text": text_aug, "label_ids": label_ids}

ds_train_sample = ds_train_sample.map(augment_text, batched=True, remove_columns=ds_train_sample.column_names).shuffle(seed=42)

In [ ]:
# embedding
import torch
from transformers import AutoTokenizer, AutoModel

model_ckpt = "miguelvictor/python-gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
  return sum_embeddings / sum_mask

def embed_text(examples):
  inputs = tokenizer(examples["text"], padding=True, truncation=True, max_length=128, return_tensors="pt")
  with torch.no_grad():
    model_output = model(**inputs)
  pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])

  return {"embedding": pooled_embeds.cpu().numpy()}

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
embs_train = ds["train"].map(embed_text, batched=True, batch_size=16)
embs_valid = ds["valid"].map(embed_text, batched=True, batch_size=16)
embs_test = ds["test"].map(embed_text, batched=True, batch_size=16)

In [ ]:
# k nearest neighbors
import faiss

embs_train.add_faiss_index("embedding")

def get_sample_preds(sample, m):
  return (np.sum(sample["label_ids"], axis=0) >= m).astype(int)

def find_best_k_m(ds_train, valid_queries, valid_labels, max_k=17):
  max_k = min(len(ds_train), max_k)
  perf_micro = np.zeros((max_k, max_k))
  perf_macro = np.zeros((max_k, max_k))

  for k in range(1, max_k):
    for m in range(1, k + 1):
      _, samples = ds_train.get_nearest_examples_batch("embedding", valid_queries, k=k)
      y_pred = np.array([get_sample_preds(s, m) for s in samples])
      clf_report = classification_report(valid_labels, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)
      perf_micro[k, m] = clf_report["micro avg"]["f1-score"]
      perf_macro[k, m] = clf_report["macro avg"]["f1-score"]
  return perf_micro, perf_macro

In [ ]:
valid_labels = np.array(embs_valid["label_ids"])
valid_queries = np.array(embs_valid["embedding"], dtype=np.float32)
perf_micro, perf_macro = find_best_k_m(embs_train, valid_queries, valid_labels)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
ax0.imshow(perf_micro)
ax1.imshow(perf_macro)

ax0.set_title("micro scores")
ax0.set_ylabel("k")
ax1.set_title("macro scores")
for ax in [ax0, ax1]:
  ax.set_xlim([0.5, 17 - 0.5])
  ax.set_ylim([17 - 0.5, 0.5])
  ax.set_xlabel("m")
plt.show()

In [ ]:
embs_train.drop_index("embedding")
test_labels = np.array(embs_test["label_ids"])
test_queries = np.array(embs_test["embedding"], dtype=np.float32)

for train_slice in train_slices:
  # create faiss
  embs_train_tmp = embs_train.select(train_slice)
  embs_train_tmp.add_faiss_index("embedding")
  # find best k, m
  perf_micro, _ = find_best_k_m(embs_train_tmp, valid_queries, valid_labels)
  k, m = np.unravel_index(perf_micro.argmax(), perf_micro.shape)
  # predictions
  _, samples = embs_train_tmp.get_nearest_examples_batch("embedding", test_queries, k=int(k))
  y_pred = np.array([get_sample_preds(s, m) for s in samples])
  # evaluate predictions
  clf_report = classification_report(test_labels, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)
  macro_scores["Embedding"].append(clf_report["macro avg"]["f1-score"])
  micro_scores["Embedding"].append(clf_report["micro avg"]["f1-score"])

In [ ]:
plot_metrics(micro_scores, macro_scores, train_samples, "Embedding")

In [ ]:
# find-tuning
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], truncation=True, max_length=128)

ds_enc = ds.map(tokenize, batched=True)
ds_enc = ds_enc.remove_columns(['labels', 'text'])

In [ ]:
ds_enc.set_format("torch")
ds_enc = ds_enc.map(lambda x: {"label_ids_f": x["label_ids"].to(torch.float)},
                    remove_columns=["label_ids"])
ds_enc = ds_enc.rename_column("label_ids_f", "label_ids")